In [37]:
import json
import numpy as np
import pandas as pd
import pickle
import torch
MOVIES_FILENAME = '../../auxiliary/movies.pkl'
FOOD_FILENAME = '../../auxiliary/food.pkl'
MUSIC_FILENAME = '../../auxiliary/music.pkl'
POI_FILENAME = '../../auxiliary/poi.pkl'
ANN_TRAIN = '../../auxiliary/raw_data/annotated_en_train.txt'
ANN_DEV = '../../auxiliary/raw_data/annotated_en_dev.txt'
ANN_TEST='../../auxiliary/raw_data/annotated_en_test.txt'

SAVE_TRAIN = '../data/processed/durecdial2_all_train_placeholder'
SAVE_DEV = '../data/processed/durecdial2_all_dev_placeholder'
SAVE_TEST= '../data/processed/durecdial2_all_test_placeholder'

In [38]:
movies_file = open(MOVIES_FILENAME, 'rb')
movies = pickle.load(movies_file)

food_file = open(FOOD_FILENAME, 'rb')
food = pickle.load(food_file) 

music_file = open(MUSIC_FILENAME,'rb')
music = pickle.load(music_file)

poi_file = open(POI_FILENAME,'rb')
poi = pickle.load(poi_file)

In [39]:
entity_id = dict()
id_entity = dict()

In [40]:
entities = set(movies)
inter = entities.intersection(food)
print(inter)
entities.update(food)
inter = entities.intersection(music)
print(inter)
entities.update(music)
inter = entities.intersection(poi)
print(inter)
entities.update(music)
entities = list(entities)

set()
{'A Chinese Ghost Story'}
set()


In [41]:
movie_id = dict()
id_movie = dict()
setter = 0
for i in range(len(movies)):
    movie_id[movies[i]] = i+1
    id_movie[i+1] = movies[i]

entity_id.update(movie_id)
id_entity.update(id_movie)
setter = len(movies)

food_id = dict()
id_food = dict()
for i in range(setter, setter + len(food)):
    food_id[food[i-setter]] = i+1
    id_food[i+1] = food[i-setter]
entity_id.update(food_id)
id_entity.update(id_food)
setter += len(food)

music_id = dict()
id_music = dict()
for i in range(setter, setter + len(music)):
    music_id[music[i-setter]] = i+1
    id_music[i+1] = music[i-setter]

entity_id.update(music_id)
id_entity.update(id_music)
setter += len(music)

poi_id = dict()
id_poi = dict()
for i in range(setter, setter + len(poi)):
    poi_id[poi[i-setter]] = i+1
    id_poi[i+1] = poi[i-setter]

entity_id.update(poi_id)
id_entity.update(id_poi)



In [42]:
print(len(movies),len(food),len(music),len(poi))


217 57 249 96


In [43]:
def process_data(filename, entity_list, entity_id, id_entity):
    file = open(filename, 'rb')
    lines = file.readlines()
    # point = json.loads(lines[0])
    # print(point['conversation'])
    conversations = []
    for line in lines:
        point = json.loads(line)
        utterances = point['conversation']
        current = []
        for i in range(len(utterances)):
            utt, seen_entities = processed_utterance(utterances[i], entity_list, entity_id, id_entity)
            utt, seen_entities = further_process(utt, seen_entities, entity_id)
            if i%2==0:
                utt = 'B: ' + utt
            else:
                utt = 'A: ' + utt
            # if utt.count('@') > 1:
            #     print(f'REPLACING    {utt}')
            #     print(f'Seen Entity   {seen_entities}')
            # # print(utt)
            current.append((utt,seen_entities))
        
        conversations.append(current)

    return conversations

def further_process(utterance, entities, entity_id):
    words = utterance.split(' ')
    new_utterance = ''
    for word in words:
        if '@' in word:
            new_utterance += '[MOVIE_ID] '
        else:
            new_utterance += word + ' '

    new_utterance = new_utterance.strip()
    if entities is None:
        return new_utterance,None

    int_entities = []
    for ent in entities:
        int_entities.append(entity_id[ent])
    
    

    return new_utterance,int_entities

def processed_utterance(utterance, entities, entity_id, id_entity):
    seen_entities = []
    for entity in entities:
        if entity in utterance:
            utterance = utterance.replace(entity, f'@{entity_id[entity]}')            
    words = utterance.split(' ')
    for word in words:
        if len(word)>0 and word[0]=='@':
            idx = 4
            for i in range(1, len(word)):
                if word[i].isnumeric()==False:
                    idx = i 
                    break
            seen_entities.append(id_entity[int(word[1:idx])])
            
                 
    if seen_entities==[]:
        return (utterance, None)

    return (utterance, seen_entities)


In [44]:
processed_train  = process_data(ANN_TRAIN, entities, entity_id, id_entity)
torch.save(processed_train,SAVE_TRAIN)

In [49]:
processed_dev  = process_data(ANN_DEV, entities, entity_id, id_entity)
torch.save(processed_dev,SAVE_DEV)

In [50]:
processed_test  = process_data(ANN_TEST, entities, entity_id, id_entity)
torch.save(processed_test,SAVE_TEST)

In [53]:
id_entity[97]

'Children From The Distant Planet'

In [54]:
entity_dictionary = {}
for id in id_entity.keys():
    entity_dictionary[id] = id_entity[id]

In [55]:
torch.save(entity_dictionary,'../data/processed/durecdial2_full_entity_db_placeholder')